https://learn.microsoft.com/en-us/azure/ai-services/agents/how-to/tools/bing-grounding?tabs=python&pivots=code-example

In [ ]:
%pip install -r requirements.txt

In [3]:
import os
from typing import Annotated

from semantic_kernel.agents.open_ai import AzureAssistantAgent
from semantic_kernel.connectors.ai.open_ai.services.azure_chat_completion import AzureChatCompletion
from semantic_kernel.contents.streaming_file_reference_content import StreamingFileReferenceContent
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole

from semantic_kernel.core_plugins.web_search_engine_plugin import WebSearchEnginePlugin

from semantic_kernel.connectors.search_engine import BingConnector
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior

from semantic_kernel.kernel import Kernel

from dotenv import load_dotenv

load_dotenv(override=True)

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_KEY")
azure_openai_chat_model = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

bing_search_key = os.getenv("BING_SEARCH_SUBSCRIPTION_KEY")

print(azure_openai_chat_model)

None


In [ ]:
kernel = Kernel()

service_id = "bing_agent"

# adding openai service to kernel
chat_service = AzureChatCompletion(
    service_id=service_id,
    api_key=azure_openai_key,
    endpoint=azure_openai_endpoint,
    deployment_name=azure_openai_chat_model,
)
kernel.add_service(chat_service)

connector = BingConnector(api_key=bing_search_key)
kernel.add_plugin(WebSearchEnginePlugin(connector), plugin_name="WebSearch")

settings = kernel.get_prompt_execution_settings_from_service_id(service_id=service_id)
settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

# creating code interpreter agent
code_agent = await AzureAssistantAgent.create(
    kernel=kernel,
    service_id=service_id,
    api_key=azure_openai_key,
    endpoint=azure_openai_endpoint,
    deployment_name=azure_openai_chat_model,
    name="BingAgent",
    instructions="""
        You are a helpful assistant that uses Bing Tool to help user find flights.
        Try to give more then one option for each request.
        Make sure to provide the flight number, airline, and price, and the source.
        Try to format your answers in mardkown table format for easy reading.
        Make sure to provide citation for the information you provide.
    """,
    execution_settings=settings,
    
    # enable_code_interpreter=True
)

print("Creating thread...")
thread_id = await code_agent.create_thread()

user_input = """
    I need help finding Long Term Assignment from New York City to London, UK
    Start Date: 2025-02-01, Duration: 3 months

    Airfare:
        Round trip, business class airfare from New York to London
        One way, business class airfare from New York to London
"""

await code_agent.add_chat_message(
    thread_id=thread_id, message=ChatMessageContent(role=AuthorRole.USER, content=user_input)
)

async for content in code_agent.invoke(thread_id=thread_id):
    print(f"# {content.role} - {content.name or '*'}: '{content.content}'")


Creating thread...
# AuthorRole.ASSISTANT - BingAgent: 'Here are some options for your flights:

### Round Trip Business Class Airfare from New York to London

| Airline            | Flight Number | Price  | Source         |
|--------------------|---------------|--------|----------------|
| Multiple Airlines  | various       | $2,299 | [Flights.com](https://flights.com) |
| Icelandair         | various       | $916   | [Priceline](https://priceline.com) |
| TAP                | various       | $1,361 | [Priceline](https://priceline.com) |

### One Way Business Class Airfare from New York to London

| Airline            | Flight Number | Price  | Source         |
|--------------------|---------------|--------|----------------|
| Icelandair         | various       | $916   | [Priceline](https://priceline.com) |
| KLM                | various       | $1,020 | [Priceline](https://priceline.com) |
| TAP                | various       | $1,361 | [Priceline](https://priceline.com) |

For book

: 